<a href="https://colab.research.google.com/github/agnesedaff/Implicit_obj_completion/blob/Script/IMPLICIT_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import transformers
from transformers import pipeline
import pandas as pd
import spacy

In [ ]:
#carico il modello per l'italiano
!python -m spacy download it_core_news_lg
nlp = spacy.load("it_core_news_lg")

In [ ]:
#imposto il modello BERT da usare con la libreria pipeline
unmasker = pipeline('fill-mask', model="dbmdz/bert-base-italian-xxl-cased", tokenizer = "dbmdz/bert-base-italian-xxl-cased")

**Primo step:** otteniamo i risultati dei modelli sul task di fill-mask

Le istruzioni per il modello sono:

1.   riempi il primo [MASK] con un Determiner e crea una nuova frase con un solo [MASK]
2.   riempi il secondo [MASK] con un Noun

In [ ]:
#importo le frasi del dataset dal file drive
file = pd.read_excel(r'/content/drive/MyDrive/IOcompletion_tesi_agnese_daffara/Dataset/IMPLICIT_results.xlsx')
sentences = file['frase']

In [ ]:
#creo due liste con gli output e i prediction scores
output = []
scores = []
#itero su ogni frase nel dataset
for sentence in sentences:
    #setto il numero di predizioni a 10
    results = unmasker(sentence, top_k=10)
    det_predictions = []
    #cerco se è disponibile un Determiner per il primo [MASK]
    for prediction in results[0]:
        #avvio l'analisi della parola con SpaCy
        doc = nlp(prediction["token_str"])
        for token in doc:
            if token.pos_ == "DET":
                det_predictions.append(prediction)
    if det_predictions:
            #aggiungo il token con POS = DET più probabile alla variabile 'det_predictions'
            det_predictions.sort(key=lambda x: x["score"], reverse=True)
            determiner = det_predictions[0]["token_str"]
            #sostituisco il Determiner al primo [MASK] e creo una nuova frase con un solo [MASK]
            new_sentence = sentence.replace("[MASK]", determiner, 1)
    else:
            new_sentence = sentence.replace("[MASK]", "", 1)

    #cerco un Noun per il secondo [MASK]
    #setto il numero di predizioni a 20
    results_new = unmasker(new_sentence, top_k=20)
    noun_predictions = []
    for prediction in results_new:
        doc = nlp(prediction['token_str'])
        for token in doc:
            if token.pos_ == "NOUN":
                noun_predictions.append(prediction)
    if noun_predictions:
        #aggiungo il token con POS = NOUN più probabile alla variabile 'noun_predictions'
        noun_predictions.sort(key=lambda x: x["score"], reverse=True)
        output.append(noun_predictions[0]["token_str"])
        #aggiungo il prediction score alla variabile 'scores'
        scores.append(round(noun_predictions[0]["score"],2))
    else:
        output.append("unknown")
        scores.append("unknown")

#stampo gli output e i prediction scores
print(output)
print(scores)